# Problem

We are a manufactoring company that sell products with credits. For every sell is made a "order".
Once a while the buyers "paid" the credit but is not always the total. We need to associate the pays with orders

In [1]:
case class Transaction[I, P](id: I, price: P)

defined class Transaction

In [2]:
val orders = (1 to 10).map(id => Transaction(id, 100 * id))
val pays = (1 to 3).map(id => Transaction(id, 850))

orders: collection.immutable.IndexedSeq[Transaction[Int, Int]] = Vector(
  Transaction(1, 100),
  Transaction(2, 200),
  Transaction(3, 300),
  Transaction(4, 400),
  Transaction(5, 500),
  Transaction(6, 600),
  Transaction(7, 700),
  Transaction(8, 800),
  Transaction(9, 900),
  Transaction(10, 1000)
)
pays: collection.immutable.IndexedSeq[Transaction[Int, Int]] = Vector(
  Transaction(1, 850),
  Transaction(2, 850),
  Transaction(3, 850)
)

In [3]:
def associate[I, P](
    pay: Transaction[I, P],
    orders: List[Transaction[I, P]],
    accs: List[Transaction[I, P]] = List(),
    debCalc: (P, List[P]) => P,
    splitCalc: (Transaction[I, P], P) => (Transaction[I, P], Transaction[I, P]),
    lessOp: (P, P) => P,
    lowerOrEquals: (P, P) => Boolean,
    zero: P,
): (List[Transaction[I, P]], List[Transaction[I, P]]) = {
    val deb = debCalc(pay.price, accs.map(_.price))
    if(lowerOrEquals(deb, zero)){
        (accs, orders)
    } else {
        val add = orders.head
        if (lowerOrEquals(add.price, deb))
            associate(
                pay,
                orders.tail,
                add :: accs,
                debCalc,
                splitCalc,
                lessOp,
                lowerOrEquals,
                zero
            )
        else {
            val (order, acc) = splitCalc(add, deb)
            associate(
                pay,
                order :: orders.tail,
                acc :: accs,
                debCalc,
                splitCalc,
                lessOp,
                lowerOrEquals,
                zero
            )
        }
    }
}

defined function associate

In [3]:
associate[Int, Int](
    pay = Transaction(1, 933),
    orders = (1 until 21).map[Int](i => Transaction(i, 100)),
    accs = List(),
    debCalc = (price, paid) => price - paid.sum,
    splitCalc = (order, price) => (order.copy(price = price), order.copy(price = order.price - price)),
    lessOp = (_ - _),
    lowerOrEquals = (_ <= _),
    0
)

cmd3.sc:3: wrong number of type parameters for method map: [B, That](f: Int => B)(implicit bf: scala.collection.generic.CanBuildFrom[scala.collection.immutable.IndexedSeq[Int],B,That])That
    orders = (1 until 21).map[Int](i => Transaction(i, 100)),
                             ^cmd3.sc:5: missing parameter type
    debCalc = (price, paid) => price - paid.sum,
               ^cmd3.sc:5: missing parameter type
    debCalc = (price, paid) => price - paid.sum,
                      ^cmd3.sc:6: missing parameter type
    splitCalc = (order, price) => (order.copy(price = price), order.copy(price = order.price - price)),
                 ^cmd3.sc:6: missing parameter type
    splitCalc = (order, price) => (order.copy(price = price), order.copy(price = order.price - price)),
                        ^cmd3.sc:7: missing parameter type for expanded function ((x$1: <error>, x$2) => x$1.$minus(x$2))
    lessOp = (_ - _),
              ^cmd3.sc:7: missing parameter type for expanded function ((x$1

: 

In [ ]:
def